# Combined YOLOv9 (Ultralytics) train, validate and detect Python implementation

## Environment preparation

### Check for Nvidia Cuda GPU

In [ ]:
!nvidia-smi

### Install python packages

In [ ]:
!pip install typeguard ultralytics

### Create folders (option: connect to Google drive for Google Colab)

In [ ]:
!mkdir detect_results train_results

In [ ]:
# Optional
from google.colab import drive

drive.mount("/content/drive/")

## Train

In [ ]:
from ultralytics import YOLO

# Parameters - https://docs.ultralytics.com/modes/train/#train-settings
data_ = "path/to/data.yaml"
model_ = "yolov9c.pt"
epochs_ = 100
time_ = None
patience_ = 100
batch_ = 16
imgsz_ = 640
save_ = True
save_period_ = -1
cache_ = False
device_ = "0"
workers_ = 8
project_ = "path/to/output/directory"
name_ = "run1"
exist_ok_ = False
pretrained_ = True
optimizer_ = "auto"
verbose_ = False
seed_ = 0
deterministic_ = True
single_cls_ = False
rect_ = False
cos_lr_ = False
close_mosaic_ = 10
resume_ = False
amp_ = True
fraction_ = 1.0
profile_ = False
freeze_ = None
lr0_ = 0.01
lrf_ = 0.01
momentum_ = 0.937
weight_decay_ = 0.0005
warmup_epochs_ = 3.0
warmup_momentum_ = 0.8
warmup_bias_lr_ = 0.1
box_ = 7.5
cls_ = 0.5
dfl_ = 1.5
pose_ = 12.0
kobj_ = 2.0
label_smoothing_ = 0.0
nbs_ = 64
overlap_mask_ = True
mask_ratio_ = 4
dropout_ = 0.0
val_ = True
plots_ = False

# Load model and train
model = YOLO(model_)
print("--- Train start ---")
model.train(data=data_, epochs=epochs_, time=time_, patience=patience_, batch=batch_, imgsz=imgsz_, save=save_,
            save_period=save_period_, cache=cache_, device=device_, workers=workers_, project=project_, name=name_,
            exist_ok=exist_ok_, pretrained=pretrained_, optimizer=optimizer_, verbose=verbose_, seed=seed_,
            deterministic=deterministic_, single_cls=single_cls_, rect=rect_, cos_lr=cos_lr_,
            close_mosaic=close_mosaic_, resume=resume_, amp=amp_, fraction=fraction_, profile=profile_,
            freeze=freeze_, lr0=lr0_, lrf=lrf_, momentum=momentum_, weight_decay=weight_decay_,
            warmup_epochs=warmup_epochs_, warmup_momentum=warmup_momentum_, warmup_bias_lr=warmup_bias_lr_,
            box=box_, cls=cls_, dfl=dfl_, pose=pose_, kobj=kobj_, label_smoothing=label_smoothing_, nbs=nbs_,
            overlap_mask=overlap_mask_, mask_ratio=mask_ratio_, dropout=dropout_, val=val_, plots=plots_)
print("--- Train end ---")

## Validate

In [ ]:
from ultralytics import YOLO

# Parameters - https://docs.ultralytics.com/modes/val/#arguments-for-yolo-model-validation
data_ = "path/to/data.yaml"
model_ = "path/to/model.pt"
batch_ = 16
imgsz_ = 640
device_ = "0"
save_json_ = False
save_hybrid_ = False
conf_ = 0.001
iou_ = 0.6
max_det_ = 300
half_ = True
dnn_ = False
plots_ = False
rect_ = False
split_ = "val"
# Load model and validate
model = YOLO(model_)
print("--- Validation start ---")
results = model.val(data=data_, imgsz=imgsz_, batch=batch_, device=device_, save_json=save_json_,
                    save_hybrid=save_hybrid_, conf=conf_, iou=iou_, max_det=max_det_, half=half_,
                    dnn=dnn_, plots=plots_, rect=rect_, split=split_)
print(results)
print("--- Validation end ---")

## Detect

In [ ]:
import os
import sys
import json

from ultralytics import YOLO

# Parameters - https://docs.ultralytics.com/modes/predict/#inference-arguments
input_path_ = "path/to/input/data/directory/or/image"
model_ = "path/to/model.pt"
output_path_ = "path/to/output/directory"
subdirectory_ = "pyexp"
config_json_name_ = "cfg.json"
conf_ = 0.25
iou_ = 0.7
imgsz_ = 640
device_ = "0"
save_cfg_ = False
save_txt_ = False
save_img_ = True
save_conf_ = False

# Prepare input data
input_data = []
is_file = os.path.isfile(input_path_)
is_dir = os.path.isdir(input_path_)
if is_file and not is_dir:
    input_data.append(input_path_)
elif is_dir and not is_file:
    for file in sorted(os.listdir(input_path_)):
        if file.endswith(".jpg") or file.endswith(".png"):
            input_data.append(os.path.join(input_path_, file))
else:
    print("Wrong input argument!")
    sys.exit()
# Load model
model = YOLO(model_)
model.info()
# Prepare output directory
output_directory_path = os.path.join(output_path_, subdirectory_)
expiter = 0
while os.path.isdir(output_directory_path):
    expiter += 1
    output_directory_path = os.path.join(output_path_, subdirectory_ + str(expiter))
os.mkdir(output_directory_path)
# Prepare json config file
config_json_path = os.path.join(output_directory_path, config_json_name_)
if save_cfg_:
    with open(config_json_path, "w") as file:
        json.dump([], file)
# Predict
results = model.predict(source=input_data, conf=conf_, iou=iou_, imgsz=imgsz_, device=device_)
# Save results
for result in results:
    image = os.path.basename(result.path)
    image_name = os.path.splitext(image)[0]
    result_txt_path = os.path.join(output_directory_path, (image_name + ".txt"))
    result_img_path = os.path.join(output_directory_path, image)
    # Prepare data
    if save_txt_:
        result.save_txt(txt_file=result_txt_path, save_conf=save_conf_)
    # Save annotated image
    if save_img_:
        result.save(filename=result_img_path)
    # Save config json file
    if save_cfg_:
        height, width = result.orig_shape
        xywh = [box for box in result.boxes.xywh.cpu().numpy().tolist()]
        xywhn = [box for box in result.boxes.xywhn.cpu().numpy().tolist()]
        cls = [cl for cl in result.boxes.cls.cpu().numpy().tolist()]
        out_obj = {
            "name": result.names,
            "speed": result.speed,
            "path": result.path,
            "xywhn": xywhn,
            "xywh": xywh,
            "cls": cls,
            "width": width,
            "height": height
        }
        with open(config_json_path, "r+") as file:
            file_data = json.load(file)
            file_data.append(out_obj)
            file.seek(0)
            json.dump(file_data, file)

#### Credits
Agnieszka Piórkowska, Miłosz Gajewski \
Politechnika Poznańska 2024